Que2.-
In this dataset you have to predict that whether we have to give ATTORNEY to the clients or not (0=Yes, 1=No) based on below factors.
CLMAGE - Age of Client
CLMSEX - Sex of client (0=Male, 1= Female)
SEATBELT - Whether client has Seatbelt or not (0=Yes, 1=No)
CLMINSUR - Whether Client give Insurance or not (0=Yes, 1=No)
LOSS - Loss that client faced

Prepare model by using logistic regression & Also calculate Accuracy by confusion matrix, Calculate accuracy score, precision score, recall score, f1 score, roc auc score. Also Visualize the result. Also select best parameters using gridsearchCV. 

Dataset : Claimants.csv

In [41]:
import warnings 
warnings.filterwarnings(action="ignore")

In [42]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns

### load the data 

In [43]:
df  = pd.read_csv("claimants.csv")
df.head(2)

,ATTORNEY,CLMSEX,CLMINSUR,SEATBELT,CLMAGE,LOSS
0,0,0.0,1.0,0.0,50.0,34.940
1,1,1.0,0.0,0.0,18.0,0.891


In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1340 entries, 0 to 1339
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   ATTORNEY  1340 non-null   int64  
 1   CLMSEX    1328 non-null   float64
 2   CLMINSUR  1299 non-null   float64
 3   SEATBELT  1292 non-null   float64
 4   CLMAGE    1151 non-null   float64
 5   LOSS      1340 non-null   float64
dtypes: float64(5), int64(1)
memory usage: 62.9 KB


In [45]:
df.isna().sum()

ATTORNEY      0
CLMSEX       12
CLMINSUR     41
SEATBELT     48
CLMAGE      189
LOSS          0
dtype: int64

In [46]:
df['CLMSEX'] = df['CLMSEX'].fillna(df['CLMSEX'].mode()[0])
df['CLMINSUR'] = df['CLMINSUR'].fillna(df['CLMINSUR'].mode()[0])
df['SEATBELT'] = df['SEATBELT'].fillna(df['SEATBELT'].mode()[0])
df['CLMAGE'] = df['CLMAGE'].fillna(df['CLMAGE'].mean())

In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1340 entries, 0 to 1339
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   ATTORNEY  1340 non-null   int64  
 1   CLMSEX    1340 non-null   float64
 2   CLMINSUR  1340 non-null   float64
 3   SEATBELT  1340 non-null   float64
 4   CLMAGE    1340 non-null   float64
 5   LOSS      1340 non-null   float64
dtypes: float64(5), int64(1)
memory usage: 62.9 KB


In [48]:
df.head(10)

,ATTORNEY,CLMSEX,CLMINSUR,SEATBELT,CLMAGE,LOSS
0,0,0.0,1.0,0.0,50.000000,34.940
1,1,1.0,0.0,0.0,18.000000,0.891
2,1,0.0,1.0,0.0,5.000000,0.330
3,0,0.0,1.0,1.0,31.000000,0.037
4,1,0.0,1.0,0.0,30.000000,0.038
5,0,1.0,1.0,0.0,35.000000,0.309
6,0,0.0,1.0,0.0,9.000000,3.538
7,0,1.0,1.0,0.0,34.000000,4.881
8,1,1.0,1.0,0.0,60.000000,0.874
9,1,0.0,1.0,0.0,28.414422,0.350


In [49]:
df['CLMSEX']=df['CLMSEX'].astype('int64')
df['CLMINSUR']=df['CLMINSUR'].astype('int64')
df['SEATBELT'] = df['SEATBELT'].astype('int64')
df['CLMAGE'] = df['CLMAGE'].astype('float64')

In [50]:
df.isnull().sum()

ATTORNEY    0
CLMSEX      0
CLMINSUR    0
SEATBELT    0
CLMAGE      0
LOSS        0
dtype: int64

In [51]:
df.isna().sum()

ATTORNEY    0
CLMSEX      0
CLMINSUR    0
SEATBELT    0
CLMAGE      0
LOSS        0
dtype: int64

In [52]:
df.describe()

,ATTORNEY,CLMSEX,CLMINSUR,SEATBELT,CLMAGE,LOSS
count,1340.000000,1340.000000,1340.000000,1340.000000,1340.000000,1340.000000
mean,0.488806,0.562687,0.910448,0.016418,28.414422,3.806307
std,0.500061,0.496240,0.285646,0.127124,18.816978,10.636903
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,1.000000,0.000000,10.000000,0.400000
50%,0.000000,1.000000,1.000000,0.000000,28.414422,1.069500
75%,1.000000,1.000000,1.000000,0.000000,40.000000,3.781500
max,1.000000,1.000000,1.000000,1.000000,95.000000,173.604000


In [53]:
df.corr()

,ATTORNEY,CLMSEX,CLMINSUR,SEATBELT,CLMAGE,LOSS
ATTORNEY,1.000000,0.076564,0.076631,-0.055848,0.009886,-0.219715
CLMSEX,0.076564,1.000000,0.065976,-0.016327,-0.022444,-0.028412
CLMINSUR,0.076631,0.065976,1.000000,0.019953,0.044706,0.004641
SEATBELT,-0.055848,-0.016327,0.019953,1.000000,-0.025379,0.111879
CLMAGE,0.009886,-0.022444,0.044706,-0.025379,1.000000,0.057641
LOSS,-0.219715,-0.028412,0.004641,0.111879,0.057641,1.000000


### split into x and y 

In [55]:
x = df.drop(['ATTORNEY', 'CLMSEX', 'CLMINSUR'], axis=1)
y = df['ATTORNEY']

### train test split 

In [56]:
from sklearn.model_selection import train_test_split

# split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(
    x, y, train_size=0.8, random_state=123456
)

In [57]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1340 entries, 0 to 1339
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   ATTORNEY  1340 non-null   int64  
 1   CLMSEX    1340 non-null   int64  
 2   CLMINSUR  1340 non-null   int64  
 3   SEATBELT  1340 non-null   int64  
 4   CLMAGE    1340 non-null   float64
 5   LOSS      1340 non-null   float64
dtypes: float64(2), int64(4)
memory usage: 62.9 KB


In [58]:
df.isna().sum()

ATTORNEY    0
CLMSEX      0
CLMINSUR    0
SEATBELT    0
CLMAGE      0
LOSS        0
dtype: int64

In [59]:
df.isnull().sum()

ATTORNEY    0
CLMSEX      0
CLMINSUR    0
SEATBELT    0
CLMAGE      0
LOSS        0
dtype: int64

In [60]:
from sklearn.linear_model import LogisticRegressionCV

model  = LogisticRegressionCV(max_iter=1000)

model.fit(x_train, y_train)



LogisticRegressionCV(max_iter=1000)

In [61]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


y_pred = model.predict(x_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
    

In [62]:
print(f"Accuracy = {accuracy}")
print(f"Precision = {precision}")
print(f"Recall = {recall}")
print(f"f1 = {f1}")

Accuracy = 0.6716417910447762
Precision = 0.6181818181818182
Recall = 0.8031496062992126
f1 = 0.6986301369863014


In [64]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

# Step 1: Define the model
model = LogisticRegression(max_iter=1000)

# Step 2: Define hyperparameter grid
parameters = {
    'penalty': ['l1', 'l2', 'elasticnet', 'none'],
    'C': [0.01, 0.1, 1, 10],
    'solver': ['liblinear', 'saga'],
    'l1_ratio': [0, 0.5, 1]  # Only used if penalty is 'elasticnet'
}

# Step 3: Create GridSearchCV object
grid_search = GridSearchCV(model, parameters, cv=5, scoring='accuracy')

# Step 4: Fit to training data
grid_search.fit(x_train, y_train)  # Use x_train and y_train here, not x_test

# Step 5: View best parameters
print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)

# Step 6: Get best estimator
best_model = grid_search.best_estimator_


Best Parameters: {'C': 0.1, 'l1_ratio': 0, 'penalty': 'l1', 'solver': 'liblinear'}
Best Score: 0.7098717670071724


In [65]:
# Step 6: Get best estimator
best_model = grid_search.best_estimator_

In [66]:
best_model

LogisticRegression(C=0.1, l1_ratio=0, max_iter=1000, penalty='l1',
                   solver='liblinear')

In [67]:
plt.figure(figsize=(10, 6))
sns.scatterplot(
    x='param_C', 
    y='mean_test_score', 
    hue='param_penalty', 
    style='param_solver', 
    data=results_df,
    s=100
)
plt.title("Scatter Plot of Grid Search Results")
plt.xlabel("C (Regularization Strength)")
plt.ylabel("Mean CV Accuracy")
plt.grid(True)
plt.show()

NameError: name 'results_df' is not defined

<Figure size 1000x600 with 0 Axes>